## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.78,147.06,69.8,52,75,18.34,broken clouds
1,1,Avarua,CK,-21.21,-159.78,75.2,83,100,9.17,overcast clouds
2,2,Ushuaia,AR,-54.80,-68.30,51.8,57,40,13.87,scattered clouds
3,3,Mataura,NZ,-46.19,168.86,77.0,41,4,6.53,clear sky
4,4,Tuktoyaktuk,CA,69.45,-133.04,12.2,85,90,9.17,light snow


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Buala,SB,-8.14,159.59,86.16,66,78,6.93,light rain
9,9,Vaini,TO,-21.20,-175.20,80.60,78,40,13.87,scattered clouds
13,13,Hilo,US,19.73,-155.09,84.20,66,40,9.17,scattered clouds
32,32,Carnarvon,AU,-24.87,113.63,82.40,32,0,17.22,clear sky
40,40,La Asuncion,VE,11.03,-63.86,89.60,62,100,14.99,overcast clouds
48,48,Airai,TL,-8.93,125.41,81.10,32,96,4.88,overcast clouds
51,51,Lorengau,PG,-2.02,147.27,85.55,72,88,2.26,light rain
68,68,Bathsheba,BB,13.22,-59.52,80.60,74,40,18.34,scattered clouds
93,93,Bengkulu,ID,-3.80,102.27,82.54,68,68,2.80,broken clouds
98,98,Bitung,ID,1.45,125.18,87.80,58,20,2.24,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                99
City                   99
Country                99
Lat                    99
Lng                    99
Max Temp               99
Humidity               99
Cloudiness             99
Wind Speed             99
Current Description    99
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Buala,SB,86.16,light rain,-8.14,159.59,
9,Vaini,TO,80.60,scattered clouds,-21.20,-175.20,
13,Hilo,US,84.20,scattered clouds,19.73,-155.09,
32,Carnarvon,AU,82.40,clear sky,-24.87,113.63,
40,La Asuncion,VE,89.60,overcast clouds,11.03,-63.86,
48,Airai,TL,81.10,overcast clouds,-8.93,125.41,
51,Lorengau,PG,85.55,light rain,-2.02,147.27,
68,Bathsheba,BB,80.60,scattered clouds,13.22,-59.52,
93,Bengkulu,ID,82.54,broken clouds,-3.80,102.27,
98,Bitung,ID,87.80,few clouds,1.45,125.18,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)  
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Buala,SB,86.16,light rain,-8.14,159.59,Maringe Lagoon Lodge
9,Vaini,TO,80.60,scattered clouds,-21.20,-175.20,Keleti Beach Resort
13,Hilo,US,84.20,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
32,Carnarvon,AU,82.40,clear sky,-24.87,113.63,Hospitality Carnarvon
40,La Asuncion,VE,89.60,overcast clouds,11.03,-63.86,Colinas de Matasiete
...,...,...,...,...,...,...,...
715,Jardim De Piranhas,BR,80.13,scattered clouds,-6.38,-37.35,AP da NIL
717,Nha Trang,VN,86.00,scattered clouds,12.25,109.18,Hanoi Golden Hotel
721,Neiafu,TO,80.60,scattered clouds,-18.65,-173.98,Mystic Sands
724,Pringsewu,ID,83.41,overcast clouds,-5.36,104.97,Urban Style by Front One


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))